In [8]:
%matplotlib widget
import os
import re
os.environ["GDAL_DATA"] = "/home/parndt/anaconda3/envs/geo_py37/share/gdal"
os.environ["PROJ_LIB"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
os.environ["PROJ_DATA"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
import h5py
import math
import datetime
import traceback
import shapely
import pytz
from tzwhere import tzwhere
from dateutil import tz
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
# from icelakes.utilities import convert_time_to_string
from IPython.display import Image, display
from matplotlib.collections import PatchCollection
from sklearn.neighbors import KDTree
import scipy as sc
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import RegularGridInterpolator
from scipy import signal
from scipy.stats import binned_statistic
from scipy.signal import find_peaks
import hdbscan
import rasterio as rio
from rasterio import plot as rioplot
from rasterio import warp
from rasterio import Affine as A
from rasterio.enums import ColorInterp
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.transform import TransformMethodsMixin
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.crs import CRS
from ipyleaflet import Map, basemaps, Polygon, GeoData, LayersControl, Polyline, Marker
from geopy.distance import geodesic
from utils import read_atl03
from utils import intersection
from utils import download_is2
from ed.edcreds import getedcreds

def get_signal(df, n_iter=100, h_thresh=[200,3], xatc_win=[500,7]):
    number_iterations = n_iter
    elevation_threshold_start = h_thresh[0]
    elevation_threshold_end = h_thresh[1]
    xatc_window_width_start = xatc_win[0]
    xatc_window_width_end = xatc_win[1]
    
    df.xatc -= df.xatc.min()
    df.xatc + np.random.uniform(low=-0.35, high=0.35, size=len(df))
    df['is_signal'] = True
    df['rmean'] = np.nan
    
    def get_params(start,end,number):
        return [start * ((end / start)**(1/(number-1))) ** it for it in range(number)]
    
    h_diff_thresholds = get_params(start=elevation_threshold_start, end=elevation_threshold_end, number=number_iterations)
    window_sizes = np.int32(np.round(get_params(start=xatc_window_width_start, end=xatc_window_width_end, number=number_iterations)))

    for i in range(number_iterations):
        rmean = df.h[df.is_signal].rolling(3*window_sizes[i],center=True,min_periods=5,win_type='gaussian').mean(
            std=int(np.ceil(window_sizes[i]/2)))
        df['rmean'] = np.interp(df.xatc, df.xatc[df.is_signal], rmean)
        df['is_signal'] = np.abs(df.rmean-df.h) < h_diff_thresholds[i]
        
    rmean = df.h[df.is_signal].rolling(5*window_sizes[i],center=True,min_periods=5,win_type='gaussian').mean(std=window_sizes[-1])
    df['rmean'] = np.interp(df.xatc, df.xatc[df.is_signal], rmean)
    
    return df

def get_oa_url(df, ancillary, gtx):
    datenum = ancillary['granule_id'][6:14]
    date = '%s-%s-%s' % (datenum[:4], datenum[4:6], datenum[6:])
    lonmin = df.lon.min()
    latmin = df.lat.min()
    lonmax = df.lon.max()
    latmax = df.lat.max()
    rgt = ancillary['rgt']
    oa_base_url = 'http://openaltimetry.org/data/api/icesat2/atl03?'
    oa_param_str = 'date={date:s}&minx={lonmin:.7f}&miny={latmin:.7f}&maxx={lonmax:.7f}&maxy={latmax:.7f}&trackId={rgt:d}&beamName={gtx:s}'
    oa_output = '&outputFormat=json'
    oa_param_str = oa_param_str.format(date=date,lonmin=lonmin,latmin=latmin,lonmax=lonmax,latmax=latmax,rgt=rgt,gtx=gtx)
    oa_url = oa_base_url + oa_param_str + oa_output
    ancillary['date'] = date
    ancillary['oa_url'] = oa_url
    print(oa_url)
    return ancillary

def get_pulseid_and_gt_latlon(df):
    df['pulseid'] = 1000*df.mframe.astype(np.uint64)+df.ph_id_pulse.astype(np.uint64)
    df['ph_index'] = df.index
    df = df.set_index('pulseid')
    thegroup = df.groupby('pulseid')
    df_grouped = thegroup[['xatc', 'lat', 'lon', 'rmean', 'h']].median()
    return df_grouped

def get_along_track_elev(df, df_grouped, xatc_resolution=0.7): 
    xatc = np.arange(df.xatc[df.is_signal].min(), df.xatc[df.is_signal].max(), xatc_resolution)
    elev = np.interp(xatc, df_grouped.xatc, df_grouped.rmean)
    lat = np.interp(xatc, df_grouped.xatc, df_grouped.lat)
    lon = np.interp(xatc, df_grouped.xatc, df_grouped.lon)
    df_out = pd.DataFrame({'lat':lat, 'lon':lon, 'xatc':xatc, 'elev_is2':elev})
    return df_out

def interpolate_dtm(terrain_model_file, elevs): 
    dtm = rio.open(terrain_model_file)
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, elevs.lon, elevs.lat)
    elevs['gt_x'] = gt_x
    elevs['gt_y'] = gt_y
    dtm_data = dtm.read(1)
    dtm_data[dtm_data==-9999] = np.nan
    _, ys = rio.transform.xy(dtm.transform, np.arange(dtm.height)+0.5, np.zeros(dtm.height))
    xs, _ = rio.transform.xy(dtm.transform, np.zeros(dtm.width), np.arange(dtm.width)+0.5)
    xs = np.array(xs)
    ys = np.flip(np.array(ys))
    X,Y = np.meshgrid(xs,ys)
    dtm_data = np.flipud(dtm_data)
    interpolator = RegularGridInterpolator((ys,xs), dtm_data, method='linear', bounds_error=False, fill_value=np.nan)
    dtm_elev_interp = interpolator((gt_y, gt_x))
    elevs['elev_dtm'] = dtm_elev_interp
    elevs = elevs[~np.isnan(elevs.elev_dtm)].copy()
    elevs['elev_diff'] = elevs.elev_is2 - elevs.elev_dtm 
    return elevs

def latlonelev2xyz(lat, lon, elevation, equatorial_radius = 6378137, flattening = 1/298.257223563): 
    # point is (lat, lon), elevation is elevation above reference ellipsoid in meters
    # equatorial_radius (in meters) and flattening correspond to the reference ellipsoid (default: WGS84)
    # implemented according to https://gssc.esa.int/navipedia/index.php/Ellipsoidal_and_Cartesian_Coordinates_Conversion
    phi = lat * (np.pi / 180)
    lamda = lon * (np.pi / 180)
    h = elevation
    f = flattening
    a = equatorial_radius
    e_squared = 2*f - f**2
    N = a / np.sqrt(1 - e_squared * np.sin(phi)**2)
    x = (N + h) * np.cos(phi) * np.cos(lamda)
    y = (N + h) * np.cos(phi) * np.sin(lamda)
    z = ((1-e_squared) * N + h) * np.sin(phi)
    return x, y, z


def xyz2latlonelev(x, y, z, equatorial_radius = 6378137, flattening = 1/298.257223563, h_tol=1e-3, lat_tol_rad=1e-6, max_iter=1e6): 
    # x, y, z are in ECEF cartesian coordinates, in meters
    # equatorial_radius (in meters) and flattening correspond to the reference ellipsoid (default: WGS84)
    # implemented according to https://gssc.esa.int/navipedia/index.php/Ellipsoidal_and_Cartesian_Coordinates_Conversion
    f = flattening
    a = equatorial_radius
    e_squared = 2*f - f**2
    p = np.sqrt(x**2 + y**2)
    lon_rad = np.arctan2(y,x)
    lon = lon_rad * (180 / np.pi)
    phi_old = np.arctan2(z, (1-e_squared)*p)
    h_old = -1e10
    h_converged = False
    phi_converged = False
    i = 0
    while not (h_converged & phi_converged): 
        N_i = a / np.sqrt(1 - e_squared * np.sin(phi_old)**2)
        h_new = p / np.cos(phi_old) - N_i
        phi_new = np.arctan2(z, ( 1 - e_squared * ( N_i / ( N_i+h_new ) ) ) * p)
        h_converged = np.abs(h_new - h_old) < h_tol
        phi_converged = np.abs(phi_new - phi_old) < lat_tol_rad
        h_old = h_new
        phi_old = phi_new
        if i > max_iter: 
            break;
        i += 1
    elev = h_new
    lat = phi_new * (180 / np.pi)
    return lat, lon, elev


def get_distance_direct(lat1, lon1, elevation1, lat2, lon2, elevation2, equatorial_radius = 6378137, flattening = 1/298.257223563):
    # point1 and point2 are (lat, lon), elevation is elevation above reference ellipsoid in meters
    x1, y1, z1 = latlonelev2xyz(lat1, lon1, elevation1, equatorial_radius, flattening)
    x2, y2, z2 = latlonelev2xyz(lat2, lon2, elevation2, equatorial_radius, flattening)

    dist = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return dist, (x1, y1, z1), (x2, y2, z2)


def get_elev(latlon, terrain_model_file=None, dtm=None, dtm_data=None):
    # get elevation of the terrain model at the lat/lon coordinates of the target point
    if (terrain_model_file is None) & ((dtm is None) | (dtm_data is None)):
        print('ERROR: No terrain model file or rasterio reader + data supplied')
        return
    if terrain_model_file is not None:
        dtm = rio.open(terrain_model_file)
        dtm_data = dtm.read(1)
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, [latlon[1]], [latlon[0]])
    dtm_data[dtm_data==-9999] = np.nan
    _, ys = rio.transform.xy(dtm.transform, np.arange(dtm.height)+0.5, np.zeros(dtm.height))
    xs, _ = rio.transform.xy(dtm.transform, np.zeros(dtm.width), np.arange(dtm.width)+0.5)
    xs = np.array(xs)
    ys = np.flip(np.array(ys))
    X,Y = np.meshgrid(xs,ys)
    dtm_data = np.flipud(dtm_data)
    interpolator = RegularGridInterpolator((ys,xs), dtm_data, method='linear', bounds_error=False, fill_value=np.nan)
    elev = interpolator((gt_y, gt_x))[0]
    return elev


def get_rgt(latlon_target, kml_filename):
    # trim the kml to a rough bounding box around the TOO coords
    latlims = [latlon_target[0]-1, latlon_target[0]+1]
    lonlims = [latlon_target[1]-1, latlon_target[1]+1]

    with open(kml_filename, 'r') as file:
        kml_string = file.read()
    kml_string = kml_string[kml_string.find('LineString_kml'):kml_string.find('</LineString>')]
    kml_coords_str = '[[' + kml_string[kml_string.find('<coordinates>')+len('<coordinates>'):kml_string.find('</coordinates>')-1] + ']]'
    kml_coords_str = kml_coords_str.replace(' ', '],[')
    kml_coords_array = np.array(eval(kml_coords_str))

    kml_lon = kml_coords_array[:,0]
    kml_lat = kml_coords_array[:,1]
    is_in_bounds_lon = (kml_lon >= lonlims[0]) & (kml_lon <= lonlims[1])
    is_in_bounds_lat = (kml_lat >= latlims[0]) & (kml_lat <= latlims[1])
    is_in_bounds = is_in_bounds_lon & is_in_bounds_lat
    kml_lon = kml_lon[is_in_bounds]
    kml_lat = kml_lat[is_in_bounds]
    return kml_lon, kml_lat


def get_closest_points_kml(kml_lat, kml_lon, too_latlon, too_elev):
    diff_df = pd.DataFrame({'lat': kml_lat, 'lon': kml_lon, 'elev': too_elev, 'dist': np.nan, 
                            'x': np.nan, 'y': np.nan, 'z': np.nan, 'idx': np.arange(len(kml_lat))})
    for i in range(len(diff_df)):
        pt = diff_df.iloc[i]
        dist, xyz1, xyz2 = get_distance_direct(lat1=pt.lat, lon1=pt.lon, elevation1=pt.elev, 
                                               lat2=too_latlon[0], lon2=too_latlon[1], elevation2=pt.elev)
        diff_df.loc[i,'dist'] = dist
        diff_df.loc[i,'x'] = xyz1[0]
        diff_df.loc[i,'y'] = xyz1[1]
        diff_df.loc[i,'z'] = xyz1[2]
    diff_df.sort_values(by='dist', inplace=True)
    diff_df = diff_df.iloc[:2].copy()
    diff_df.sort_values(by='idx', inplace=True)
    return diff_df


def get_closest_point(closest_points_kml_df, too_latlon, too_elev, n_pts=10000):
    n_pts = 10000
    lats_ = np.linspace(closest_points_kml_df.iloc[0].lat, closest_points_kml_df.iloc[1].lat, n_pts)
    lons_ = np.linspace(closest_points_kml_df.iloc[0].lon, closest_points_kml_df.iloc[1].lon, n_pts)
    diff_df = pd.DataFrame({'lat': lats_, 'lon': lons_, 'elev': too_elev, 'dist': np.nan, 
                            'x': np.nan, 'y': np.nan, 'z': np.nan})
    for i in range(len(diff_df)):
        pt = diff_df.iloc[i]
        dist, xyz1, xyz2 = get_distance_direct(lat1=pt.lat, lon1=pt.lon, elevation1=pt.elev, 
                                               lat2=too_latlon[0], lon2=too_latlon[1], elevation2=pt.elev)
        diff_df.loc[i,'dist'] = dist
        diff_df.loc[i,'x'] = xyz1[0]
        diff_df.loc[i,'y'] = xyz1[1]
        diff_df.loc[i,'z'] = xyz1[2]
    diff_df.sort_values(by='dist', inplace=True)
    closest_point = diff_df.iloc[0].copy()
    return closest_point


def unit_vector(vector):
    return vector / np.linalg.norm(vector)


def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


def get_gt_on_terrain(start, direction, dtm, dtm_data, fit_to_zero=False, disp=False):
    
    def line_func(t):
        return start + t*direction
    
    def target_func(t):
        p_line = line_func(t)
        lat, lon, elev = xyz2latlonelev(*p_line, h_tol=1e-2, lat_tol_rad=1e-4)
        if fit_to_zero:
            elev_surface = 0.0
        else:
            elev_surface = get_elev((lat,lon), dtm=dtm, dtm_data=dtm_data)
            if np.isnan(elev_surface): 
                elev_surface = np.nanmean(dtm_data[dtm_data>-1000])
        elev_diff = elev-elev_surface
        return np.sum((elev - elev_surface)**2)
    
    t_opt = sc.optimize.fmin(target_func, x0=1.0, disp=disp);
    intersection_point = xyz2latlonelev(*line_func(t_opt))
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, [intersection_point[1]], [intersection_point[0]])
    return (intersection_point, (gt_x[0], gt_y[0]))


def add_timestamps(kml_df, kml_file, too_latlon, local_time_zone=None):
    with open(kml_file, 'r') as file:
            kml_string = file.read()
    kml_string.find('<Point id=')
    starts = [m.start() for m in re.finditer('<Point id=', kml_string)]
    ends = [m.start()+100 for m in re.finditer('</Point>', kml_string)]
    stringlist = [kml_string[s:e] for s,e in list(zip(starts, ends))]
    lats, lons, dts_utc, dts_local, dtstmp_utc, dtstmp_local, str_utc, str_local = [], [], [], [], [], [], [], []
    for substr in stringlist:
        coordstr = substr[substr.find('<coordinates>')+len('<coordinates>'):substr.find('</coordinates>')]
        coordstr = '['+coordstr[:coordstr.rfind(',')-1]+']'
        coords = eval(coordstr)
        lat = coords[1]
        lon = coords[0]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        dt_str
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        day = int(dt_str[:dt_str.find('-')])
        month_abbr = dt_str[dt_str.find('-')+1:dt_str.rfind('-')]
        year = int(dt_str[dt_str.rfind('-')+1:dt_str.find(' ')])
        hrs = int(dt_str[dt_str.find(' ')+1:dt_str.find(':')])
        mins = int(dt_str[dt_str.find(':')+1:dt_str.rfind(':')])
        secs = int(dt_str[dt_str.rfind(':')+1:])
        datetime_str = '%4i-%3s-%02iT%02i:%02i:%02iZ' % (year, month_abbr, day, hrs, mins, secs)
        dt = datetime.datetime.strptime(datetime_str,'%Y-%b-%dT%H:%M:%SZ')
        timestamp_utc = datetime.datetime.timestamp(dt)
        from_zone = tz.gettz('UTC')
        to_zone = tz.gettz(local_time_zone)
        if local_time_zone is None:
            tzw = tzwhere.tzwhere()
            local_time_zone = tzw.tzNameAt(coords[1], coords[0])
            to_zone = tz.gettz(local_time_zone)
        dt_local = dt.astimezone(to_zone)
        timedelta_local = dt_local.utcoffset()
        lats.append(lat)
        lons.append(lon)
        dtstmp_utc.append(timestamp_utc)
    df = pd.DataFrame({'lat':lats, 'lon':lons, 'dtstmp_UTC': dtstmp_utc})
    near_lat = (df.lat >= too_latlon[0]-10) & (df.lat <= too_latlon[0]+10)
    near_lon = (df.lon >= too_latlon[1]-10) & (df.lon <= too_latlon[1]+10)
    df = df[near_lat & near_lon]
    dtstmp_utc_interp = np.interp(kml_df.lat, df.lat, df.dtstmp_UTC)
    kml_df['dtstmp_utc'] = dtstmp_utc_interp
    return kml_df, timedelta_local


# get the actual laser footprint tracks on the ground
def get_footprints(dtm, closest_points_kml_df, submission_point_dtm, spacecraft_orientation, ground_track_buffer,
                  ground_track_resolution, sc_closest_xyz, beam_strength, local_crs_meters):
    xs, ys = warp.transform({'init': 'epsg:4326'}, local_crs_meters, np.array(closest_points_kml_df.lon), np.array(closest_points_kml_df.lat))
    dx = xs[1] - xs[0]
    dy = ys[1] - ys[0]
    normal_vector = [-dy, dx] / np.sqrt(dy**2 + dx**2)
    x_left = submission_point_dtm[0] + normal_vector[0] * 45
    y_left = submission_point_dtm[1] + normal_vector[1] * 45
    x_right = submission_point_dtm[0] - normal_vector[0] * 45
    y_right = submission_point_dtm[1] - normal_vector[1] * 45
    lons, lats = warp.transform(local_crs_meters, {'init': 'epsg:4326'}, np.array([x_left,x_right]), np.array([y_left, y_right]))
    closest_points_beams_latlonelev = {'left': (lats[0], lons[0], 0.0), 'right': (lats[1], lons[1], 0.0)}
    closest_points_beams_xyzs = {k:np.array(latlonelev2xyz(*v)) for (k,v) in closest_points_beams_latlonelev.items()}
    p1_latlonelev = (closest_points_kml_df.iloc[0].lat, closest_points_kml_df.iloc[0].lon, closest_points_kml_df.iloc[0].elev)
    p2_latlonelev = (closest_points_kml_df.iloc[1].lat, closest_points_kml_df.iloc[1].lon, closest_points_kml_df.iloc[1].elev)
    p1_xyz = np.array(latlonelev2xyz(*p1_latlonelev))
    p2_xyz = np.array(latlonelev2xyz(*p2_latlonelev))
    flight_direction = unit_vector(p2_xyz - p1_xyz)
    sc_offsets_before = -np.arange(0, np.abs(ground_track_buffer[0])+1e-5, ground_track_resolution)
    sc_offsets_after = np.arange(0, np.abs(ground_track_buffer[1])+1e-5, ground_track_resolution)
    offsets_sc = np.sort(np.concatenate((sc_offsets_before, sc_offsets_after[1:])))
    sc_positions_xyz = [sc_closest_xyz + offset * flight_direction for offset in offsets_sc]
    footprints_dict = {}
    for side in beam_strength.keys():
        direction = closest_points_beams_xyzs[side] - sc_closest_xyz
        intersection_points, gt_xys_dtm_crs = [], []
        print('Calculating laser footprint track on the topography (%s side, %s beam)' % (side, beam_strength[side]))
        for i, start_point in enumerate(sc_positions_xyz):
            intersection_point, gt_xy = get_gt_on_terrain(start_point, direction, dtm=dtm, dtm_data=dtm_data, fit_to_zero=False, disp=False)
            intersection_points.append(intersection_point)
            gt_xys_dtm_crs.append(gt_xy)
            print('---> calculated %4i / %4i points' % (i+1,len(sc_positions_xyz)), end='\r')
        print('---> DONE!                                                                        ')
        df_footprints = pd.DataFrame({'lat': np.array(intersection_points)[:,0],
                                      'lon': np.array(intersection_points)[:,1],
                                      'elev': np.array(intersection_points)[:,2],
                                      'rgt_x': np.array(gt_xys_dtm_crs)[:,0],
                                      'rgt_y': np.array(gt_xys_dtm_crs)[:,1],
                                     })
        footprints_dict[side] = df_footprints
    return footprints_dict

def plot_over_dtm(dtm_viz_file, too_latlon, submission_point_dtm, footprints_dict, beam_strength):
    viz = rio.open(dtm_viz_file)
    fig, ax = plt.subplots(figsize=[7, 7], dpi=100)
    plt.rcParams.update({'font.size': 7})
    img_plot = rioplot.show(viz, ax=ax)
    too_x_viz, too_y_viz = warp.transform({'init': 'epsg:4326'},viz.crs, [too_latlon[1]], [too_latlon[0]])
    ax.plot(too_x_viz, too_y_viz, 'ro', label='TOO target point', ms=2)
    ax.plot(submission_point_dtm[0], submission_point_dtm[1], 'bo', label='TOO submission point', ms=2)
    for side in footprints_dict.keys():
        beamstrength = beam_strength[side]
        df_footprints = footprints_dict[side]
        ls = '-' if beamstrength=='strong' else '--'
        ax.plot(df_footprints.rgt_x, df_footprints.rgt_y, c='k', lw=0.5, marker='o', ms=1, ls=ls,
                label='TOO center %s beam\nfootprint track on topography' % beamstrength)
    ax.legend(loc='upper right')
    ax.set_xlim((viz.bounds.left, viz.bounds.right))
    ax.set_ylim((viz.bounds.bottom, viz.bounds.top))
    return fig

def plot_leaflet(kml_lat, kml_lon, too_latlon, too_submission_point, closest_point, footprints_dict, beam_strength):
    m=Map(basemap=basemaps.Esri.WorldImagery, center=too_latlon, zoom=11)
    line_loc_list = list(zip(kml_lat.T,kml_lon.T))
    line = Polyline(locations=line_loc_list,color="green",weight=3,fill=False)
    m.add_layer(line)
    marker_too = Marker(location=too_latlon, draggable=False, title='TOO target')
    m.add_layer(marker_too);
    marker_too_sub = Marker(location=too_submission_point, draggable=False, title='TOO submission point')
    m.add_layer(marker_too_sub);
    closest_point_plot = (closest_point.lat,closest_point.lon)
    marker_closest_point = Marker(location=closest_point_plot, draggable=False, opacity=0.7, title='closest point on RGT')
    m.add_layer(marker_closest_point);
    connector = Polyline(locations=[too_latlon, closest_point_plot],color="blue", weight=1, fill=False)
    m.add_layer(connector)
    for side in footprints_dict.keys():
        beamstrength = beam_strength[side]
        df_footprints = footprints_dict[side]
        thisweight = 2 if beamstrength=='strong' else 1
        line_loc_list = list(zip(df_footprints.lat.T,df_footprints.lon.T))
        thisline = Polyline(locations=line_loc_list,color="black",weight=thisweight,fill=False)
        m.add_layer(thisline)
    return m

In [9]:
## run to download the granule
# dtm_bbox = [[520925.877649,525919.746883],[3766456.711576,3773181.723345]]
# coords_latlon = warp.transform({'init': 'epsg:32611'}, {'init': 'epsg:4326'}, dtm_bbox[0], dtm_bbox[1])
boundbox = list(np.array(coords_latlon).T.flatten())
start_date = '2021-09-27'
end_date = '2021-09-27'
output_dir = 'data/IS2/atl03/sang_test_veg_offpointing'
uid, pwd, email = getedcreds()
download_is2(start_date=start_date, end_date=end_date, uid=uid, pwd=pwd, bbox=boundbox, output_dir=output_dir)

In [17]:
boundbox

[-116.77329921437698,
 34.03857886135635,
 -116.71899841669249,
 34.09911814342687]

In [10]:
spacecraft_elevation = 484e3 # meters (roughly)
spacecraft_orientation = 'forward' # from TechRefTable20230307
granule_name = 'ATL03_20210927194224_00821302_005_01.h5'
too_submit_latlonelev = np.array((34.06683584237977, -116.7476751616571, 0.0)) # from the cycle 13 KML
kml_file_thiscycle = 'data/IS2/kmls/IS2_RGT_0082_cycle13_27-Sep-2021.kml' # includes veg off-pointing -> use RGT pointing KML for geometry instead
kml_file_RGTpointingcycle = 'data/IS2/kmls/IS2_RGT_0082_cycle12_28-Jun-2021.kml' # cycle 12 points to RGT, so represents sub-satellite point
atl03_file = 'data/IS2/atl03/sang_test_veg_offpointing/processed_ATL03_20210927194224_00821302_005_01.h5'
terrain_model_file = 'data/LiDAR/vegpointing_test_sang/output_be.tif'
dtm_viz_file = 'data/LiDAR/vegpointing_test_sang/viz.be_hillshade-color.tif'
local_crs_meters = CRS.from_epsg(32611)
#img_file = 'data/imagery/COPERNICUS-S2_SR-20210409T175909_20210409T180701_T12SYH-8bitRGB.tif'

local_time_zone = 'America/Los_Angeles'
ground_track_buffer = (-3000, 3000) # meters
ground_track_resolution = 1000

along_track_resolution = 1.0
gtx_list = ['gt2l', 'gt2r']
dtm = rio.open(terrain_model_file)
dtm_data = dtm.read(1)

too_elev = get_elev(too_submit_latlonelev[:2], dtm=dtm, dtm_data=dtm_data)
kml_lon, kml_lat = get_rgt(too_submit_latlonelev[:2], kml_file_RGTpointingcycle)
kml_df = pd.DataFrame({'lon': kml_lon, 'lat': kml_lat})
kml_df, timedelta_localtime = add_timestamps(kml_df, kml_file_thiscycle, too_submit_latlonelev[:2], local_time_zone=local_time_zone)
closest_points_kml_df =  get_closest_points_kml(kml_lat, kml_lon, too_submit_latlonelev[:2], too_elev)
closest_point = get_closest_point(closest_points_kml_df, too_submit_latlonelev[:2], too_elev, n_pts=10000)

In [11]:
sc_closest_latlonelev = (closest_point.lat, closest_point.lon, spacecraft_elevation)
sc_closest_xyz = np.array(latlonelev2xyz(*sc_closest_latlonelev))
closest_point_rgt_xyz = np.array((closest_point.x, closest_point.y, closest_point.z))
too_target_xyz = np.array(latlonelev2xyz(too_submit_latlonelev[0], too_submit_latlonelev[1], 0.0))
off_point_angle_rad = angle_between(sc_closest_xyz-closest_point_rgt_xyz, sc_closest_xyz-too_target_xyz)
off_point_angle_deg = off_point_angle_rad * 180 / np.pi
direction = too_target_xyz - sc_closest_xyz
start = sc_closest_xyz
intersection_point, xy_dtm_target = get_gt_on_terrain(start, direction, dtm=dtm, dtm_data=dtm_data, fit_to_zero=False, disp=False)
too_latlon = intersection_point[:2]
time_utc_closest = datetime.datetime.fromtimestamp(np.interp(closest_point.lat, kml_df.lat, kml_df.dtstmp_utc))
time_local_closest = time_utc_closest + timedelta_localtime
time_utc_string = datetime.datetime.strftime(time_utc_closest,'%Y-%b-%dT%H:%M:%SZ')
time_local_string = datetime.datetime.strftime(time_local_closest,'%Y-%b-%d %H:%M:%S') + (' %s' % local_time_zone)
offset_distance = geodesic((too_latlon[0], too_latlon[1]), (too_submit_latlonelev[0], too_submit_latlonelev[1])).m
offset_distance_approx = np.tan(off_point_angle_rad) * too_elev
f = h5py.File(atl03_file, 'r')
actual_offset_angle = np.mean(90 - 180*np.array(f['gt2l']['geolocation']['ref_elev'])/np.pi)

In [12]:
print('KML used for RGT : %s' % kml_file_RGTpointingcycle)
print('KML used for time of overpass : %s' % kml_file_thiscycle)
print('ICESat-2 data granule : %s' % granule_name)
print('Digital Terrain Model used : %s' % terrain_model_file)
print('')
print('Off-pointing angle estimate          : %.4f degrees' % off_point_angle_deg)
print('Off-pointing angle actual from ATL03 : %.4f degrees' % actual_offset_angle)
print('TOO calculated coordinates on topography : %11.7f, %12.7f (%7.2f m above WGS84)' % (too_latlon[0], too_latlon[1], too_elev))
print('TOO submission coordinates               : %11.7f, %12.7f (%7.2f m above WGS84)' % tuple(too_submit_latlonelev))
print('Offset distance between submission and calculated coordinates on topography : %6.1f m' % offset_distance)
print('Offset distance estimated by tan(pointing_angle)*TOO_elevation              : %6.1f m' % offset_distance_approx)
print('')
print('Spacecraft elevation estimate used : %.3f km' % (spacecraft_elevation/1000))
print('Spacecraft orientation from TechRefTable : %s ' % spacecraft_orientation)
print('Estimated overpass time: %s (%s)' % (time_utc_string, time_local_string))
print('Closest point on RGT : %.7f, %.7f' % (closest_point.lat, closest_point.lon))
print('Off-pointing distance from sub_satellite point at topography elevation of TOO : %.2f km' % (closest_point.dist/1000))

KML used for RGT : data/IS2/kmls/IS2_RGT_0082_cycle12_28-Jun-2021.kml
KML used for time of overpass : data/IS2/kmls/IS2_RGT_0082_cycle13_27-Sep-2021.kml
ICESat-2 data granule : ATL03_20210927194224_00821302_005_01.h5
Digital Terrain Model used : data/LiDAR/vegpointing_test_sang/output_be.tif

Off-pointing angle estimate          : 1.2337 degrees
Off-pointing angle actual from ATL03 : 1.3856 degrees
TOO calculated coordinates on topography :  34.0668045, -116.7480699 (1581.58 m above WGS84)
TOO submission coordinates               :  34.0668358, -116.7476752 (   0.00 m above WGS84)
Offset distance between submission and calculated coordinates on topography :   36.6 m
Offset distance estimated by tan(pointing_angle)*TOO_elevation              :   34.1 m

Spacecraft elevation estimate used : 484.000 km
Spacecraft orientation from TechRefTable : backward 
Estimated overpass time: 2021-Sep-27T19:48:55Z (2021-Sep-27 12:48:55 America/Los_Angeles)
Closest point on RGT : 34.0578585, -116.860064

In [13]:
beam_strength = {'left': 'strong', 'right': 'weak'} if spacecraft_orientation == 'forward' else {'left': 'weak', 'right': 'strong'}
submission_point_local = warp.transform({'init': 'epsg:4326'}, local_crs_meters, [too_submit_latlonelev[1]],[too_submit_latlonelev[0]])
footprints_dict = get_footprints(dtm, closest_points_kml_df, submission_point_local, spacecraft_orientation, 
                                 ground_track_buffer, ground_track_resolution, sc_closest_xyz, beam_strength, local_crs_meters)

Calculating laser footprint track on the topography (left side, weak beam)
---> DONE!                                                                        
Calculating laser footprint track on the topography (right side, strong beam)
---> DONE!                                                                        


In [14]:
is2dict = {}
for gtx in gtx_list:
    gtxs, ancillary, photon_data = read_atl03(atl03_file, geoid_h=False, gtxs_to_read=gtx)
    df_photon = get_signal(photon_data[gtx])
    ancillary = get_oa_url(df_photon, ancillary, gtx)
    df_grouped = get_pulseid_and_gt_latlon(df_photon[df_photon.is_signal].copy())
    df_elevs = get_along_track_elev(df_photon, df_grouped, xatc_resolution=along_track_resolution)
    df_elevs = interpolate_dtm(terrain_model_file, df_elevs)
    is2dict[gtx] = df_elevs

  reading in data/IS2/atl03/sang_test_veg_offpointing/processed_ATL03_20210927194224_00821302_005_01.h5
  reading in beam: gt2l  --> done.
http://openaltimetry.org/data/api/icesat2/atl03?date=2021-09-27&minx=-116.7519391&miny=34.0385442&maxx=-116.7446446&maxy=34.0991827&trackId=82&beamName=gt2l&outputFormat=json
  reading in data/IS2/atl03/sang_test_veg_offpointing/processed_ATL03_20210927194224_00821302_005_01.h5
  reading in beam: gt2r  --> done.
http://openaltimetry.org/data/api/icesat2/atl03?date=2021-09-27&minx=-116.7511133&miny=34.0386086&maxx=-116.7436853&maxy=34.0990444&trackId=82&beamName=gt2r&outputFormat=json


In [15]:
m=Map(basemap=basemaps.Esri.WorldImagery, center=too_latlon, zoom=11)
line_loc_list = list(zip(kml_lat.T,kml_lon.T))
line = Polyline(locations=line_loc_list,color="green",weight=3,fill=False)
m.add_layer(line)
marker_too = Marker(location=too_latlon, draggable=False, title='TOO target')
m.add_layer(marker_too);
marker_too_sub = Marker(location=list(too_submit_latlonelev[:2]), draggable=False, title='TOO submission point')
m.add_layer(marker_too_sub);
closest_point_plot = (closest_point.lat,closest_point.lon)
marker_closest_point = Marker(location=closest_point_plot, draggable=False, opacity=0.7, title='closest point on RGT')
m.add_layer(marker_closest_point);
connector = Polyline(locations=[too_latlon, closest_point_plot],color="blue", weight=1, fill=False)
m.add_layer(connector)
for side in footprints_dict.keys():
    beamstrength = beam_strength[side]
    df_footprints = footprints_dict[side]
    thisweight = 2 if beamstrength=='strong' else 1
    line_loc_list = list(zip(df_footprints.lat.T,df_footprints.lon.T))
    thisline = Polyline(locations=line_loc_list,color="black",weight=thisweight,fill=False)
    m.add_layer(thisline)
m

Map(center=[34.066804486415386, -116.74806987878793], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [16]:
fig,ax = plt.subplots(figsize=[6, 6], dpi=100)
plt.rcParams.update({'font.size': 5})
viz = rio.open(dtm_viz_file)
img_plot = rioplot.show(viz, ax=ax)
submission_point_dtm = warp.transform({'init': 'epsg:4326'},viz.crs, [too_submit_latlonelev[1]],[too_submit_latlonelev[0]])
ax.plot(submission_point_dtm[0], submission_point_dtm[1], 'bo', label='Veg off-pointing target', ms=3)
calc_surf_point_dtm = warp.transform({'init': 'epsg:4326'},viz.crs,[too_latlon[1]], [too_latlon[0]])
ax.plot(calc_surf_point_dtm[0], calc_surf_point_dtm[1], 'ko', label='calculated actual point on surface', ms=3)
for side in footprints_dict.keys():
    beamstrength = beam_strength[side]
    df_footprints = footprints_dict[side]
    ls = '-' if beamstrength=='strong' else '--'
    ax.plot(df_footprints.rgt_x, df_footprints.rgt_y, c='k', lw=0.5, marker='o', ms=1, ls=ls,
            label='TOO center %s beam\nfootprint track on topography' % beamstrength)
for gtx in gtx_list:
    beamstrength = ancillary['gtx_strength_dict'][gtx]
    df = is2dict[gtx]
    ls = '-' if beamstrength=='strong' else '--'
    ax.plot(df.gt_x, df.gt_y, c='r', lw=0.5, ls=ls,
                label='actual ICESat-2 locations (%s, %s)' % (gtx,beamstrength))
    
ax.legend(loc='upper right')
#ax.set_aspect('auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
too_latlon = (37.9070400, -107.7263000)

spacecraft_elevation = 480e3 # meters (roughly)
spacecraft_orientation = 'backward' # from TechRefTable20230307
granule_name = 'ATL03_20210409031439_02341102_005_01.h5'
kml_file = 'data/IS2/kmls/IS2_RGT_0234_cycle11_09-Apr-2021.kml'
atl03_file = 'data/IS2/atl03/processed_ATL03_20210409031439_02341102_005_01.h5'
terrain_model_file = 'data/LiDAR/CO_TOO_validation/output_USGS10m.tif'
dtm_viz_file = 'data/LiDAR/CO_TOO_validation/viz.USGS10m_hillshade-color.tif'
img_file = 'data/imagery/COPERNICUS-S2_SR-20210409T175909_20210409T180701_T12SYH-8bitRGB.tif'

local_time_zone = 'America/Denver'
ground_track_buffer = (-3000, 3000) # meters
ground_track_resolution = 1000

In [ ]:
dtm = rio.open(terrain_model_file)
dtm_data = dtm.read(1)
too_elev = get_elev(too_latlon, dtm=dtm, dtm_data=dtm_data)
kml_lon, kml_lat = get_rgt(too_latlon, kml_file)
kml_df = pd.DataFrame({'lon': kml_lon, 'lat': kml_lat})
kml_df, timedelta_localtime = add_timestamps(kml_df, kml_file, too_latlon, local_time_zone=local_time_zone)
closest_points_kml_df =  get_closest_points_kml(kml_lat, kml_lon, too_latlon, too_elev)
closest_point = get_closest_point(closest_points_kml_df, too_latlon, too_elev, n_pts=10000)

In [ ]:
sc_closest_latlonelev = (closest_point.lat, closest_point.lon, spacecraft_elevation)
sc_closest_xyz = np.array(latlonelev2xyz(*sc_closest_latlonelev))
closest_point_rgt_xyz = np.array((closest_point.x, closest_point.y, closest_point.z))
too_target_xyz = np.array(latlonelev2xyz(too_latlon[0], too_latlon[1], too_elev))
off_point_angle_rad = angle_between(sc_closest_xyz-closest_point_rgt_xyz, sc_closest_xyz-too_target_xyz)
off_point_angle_deg = off_point_angle_rad * 180 / np.pi
direction = too_target_xyz - sc_closest_xyz
start = sc_closest_xyz
intersection_point, submission_point_dtm = get_gt_on_terrain(start, direction, dtm=dtm, dtm_data=dtm_data, fit_to_zero=True, disp=False)
too_submission_point = intersection_point[:2]
time_utc_closest = datetime.datetime.fromtimestamp(np.interp(closest_point.lat, kml_df.lat, kml_df.dtstmp_utc))
time_local_closest = time_utc_closest + timedelta_localtime
time_utc_string = datetime.datetime.strftime(time_utc_closest,'%Y-%b-%dT%H:%M:%SZ')
time_local_string = datetime.datetime.strftime(time_local_closest,'%Y-%b-%d %H:%M:%S') + (' %s' % local_time_zone)

print('KML used for RGT : %s' % kml_file)
print('Digital Terrain Model used : %s' % terrain_model_file)
print('TOO target coordinates on topography : %11.7f, %12.7f (%7.2f m above WGS84)' % (too_latlon[0], too_latlon[1], too_elev))
print('TOO submission coordinates           : %11.7f, %12.7f (%7.2f m above WGS84)' % (too_submission_point[0], too_submission_point[1], 0.0))
print('Off-pointing angle estimate : %.4f degrees.' % off_point_angle_deg)
print('Off-pointing distance at topography elevation of TOO : %.2f km' % (closest_point.dist/1000))
print('Closest point on RGT : %.7f, %.7f' % (closest_point.lat, closest_point.lon))
print('Estimated overpass time: %s (%s)' % (time_utc_string, time_local_string))

In [ ]:
beam_strength = {'left': 'weak', 'right': 'strong'} if spacecraft_orientation == 'forward' else {'left': 'strong', 'right': 'weak'}
img = rio.open(img_file)
submission_point_img = warp.transform({'init': 'epsg:4326'}, img.crs, [too_submission_point[1]],[too_submission_point[0]])
footprints_dict = get_footprints(dtm, closest_points_kml_df, submission_point_img, spacecraft_orientation, 
                               ground_track_buffer, ground_track_resolution, sc_closest_xyz, beam_strength, img)

In [ ]:
fig = plot_over_dtm(dtm_viz_file, too_latlon, submission_point_dtm, footprints_dict, beam_strength)

In [ ]:
themap = plot_leaflet(kml_lat, kml_lon, too_latlon, too_submission_point, closest_point, footprints_dict, beam_strength)
themap